In [ ]:
# Active Vision Uncertainty Reward - Sanity Checks

This notebook demonstrates the basic functionality of the active vision uncertainty reward system. We'll test the entropy computation, heatmap generation, and visualization capabilities.

## Overview

The system implements information-theoretic uncertainty measures for active vision tasks by:
1. Computing Shannon entropy from Vision Transformer features
2. Generating per-pixel uncertainty heatmaps
3. Providing viewpoint scanning capabilities for robotic vision

Let's start by importing the necessary libraries and testing the core functionality.


In [ ]:
import sys
import os
sys.path.append('../src')

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import our uncertainty modules
from uncertainty.core import rgb_entropy, save_heatmap, UncertaintyModel
from uncertainty.scan_views import generate_synthetic_image

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


In [ ]:
## Test 1: Basic Entropy Computation

Let's start by testing the core entropy computation functionality with a random RGB image:


In [ ]:
# Create a random RGB image
rgb_tensor = torch.rand(3, 224, 224)
print(f"Input image shape: {rgb_tensor.shape}")
print(f"Value range: [{rgb_tensor.min():.3f}, {rgb_tensor.max():.3f}]")

# Compute entropy
entropy, heatmap = rgb_entropy(rgb_tensor)

print(f"\nEntropy computation results:")
print(f"Scalar entropy: {entropy:.4f}")
print(f"Heatmap shape: {heatmap.shape}")
print(f"Heatmap range: [{heatmap.min():.3f}, {heatmap.max():.3f}]")
print(f"Heatmap mean: {heatmap.mean():.4f}")
print(f"Heatmap std: {heatmap.std():.4f}")

# Visualize the input image and heatmap
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original image
rgb_numpy = rgb_tensor.permute(1, 2, 0).numpy()
axes[0].imshow(rgb_numpy)
axes[0].set_title('Original Random Image')
axes[0].axis('off')

# Heatmap
im1 = axes[1].imshow(heatmap, cmap='viridis')
axes[1].set_title(f'Uncertainty Heatmap (entropy={entropy:.4f})')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1], label='Entropy')

# Overlay
axes[2].imshow(rgb_numpy)
axes[2].imshow(heatmap, alpha=0.6, cmap='viridis')
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
## Test 2: Synthetic Image Generation

Let's test the synthetic image generation functionality used in viewpoint scanning:


In [ ]:
# Generate synthetic images from different viewpoints
poses = [
    np.array([1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]),  # Front view
    np.array([0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]),  # Side view
    np.array([-1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]), # Back view
    np.array([0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0])   # Top view
]

view_names = ['Front', 'Side', 'Back', 'Top']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, (pose, name) in enumerate(zip(poses, view_names)):
    # Generate synthetic image
    synthetic_image = generate_synthetic_image(pose)
    
    # Compute entropy
    entropy, heatmap = rgb_entropy(synthetic_image)
    
    # Convert to numpy for visualization
    image_numpy = synthetic_image.permute(1, 2, 0).numpy()
    
    # Plot original image
    axes[0, i].imshow(image_numpy)
    axes[0, i].set_title(f'{name} View\nPose: ({pose[0]:.1f}, {pose[1]:.1f}, {pose[2]:.1f})')
    axes[0, i].axis('off')
    
    # Plot heatmap
    im = axes[1, i].imshow(heatmap, cmap='viridis')
    axes[1, i].set_title(f'Entropy: {entropy:.4f}')
    axes[1, i].axis('off')
    
    print(f"{name} view - Entropy: {entropy:.4f}")

plt.tight_layout()
plt.show()


In [ ]:
## Summary

This notebook successfully demonstrated:

1. **Core Entropy Computation**: The `rgb_entropy` function correctly computes Shannon entropy from RGB images using Vision Transformer features and returns both scalar entropy values and spatial heatmaps.

2. **Heatmap Generation**: The uncertainty heatmaps show meaningful spatial patterns of uncertainty across the image, with values properly normalized and finite.

3. **Synthetic Image Generation**: The viewpoint-dependent synthetic image generation creates visually distinct images based on camera poses, enabling systematic testing of the uncertainty computation.

4. **End-to-End Workflow**: The complete pipeline from image generation to entropy computation to visualization works as expected.

The system is ready for integration with RLBench and deployment in active vision scenarios. The entropy values and heatmaps provide meaningful uncertainty information that can guide robotic vision systems in selecting optimal viewpoints.
